In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
from collections import defaultdict
import re
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd   

# Analisis Tf-idf y NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text 

from gensim import corpora, models, similarities, matutils
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.matutils import cossim
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import pickle
import random
import sys
from tqdm import tqdm
from sklearn import manifold


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
TN= pd.read_csv('TN_final.csv', sep= ';')
#LPO= pd.read_csv('lpo.csv', error_bad_lines=False)
#P12= pd.read_csv('p12.csv', error_bad_lines=False)
TN.head()

,Unnamed: 0,cuerpo,fecha,seccion,titulo
0,0,"[<p>El ex jugador de Boca, ahora columnista en...",12/11/2018,deportes,El fuerte cruce entre Raúl Cascini y un period...
1,1,[<p>El gran ícono de los cómics estadounidense...,12/11/2018,tecno,"Los ""hijos"" de Stan Lee: de Spiderman a Hulk, ..."
2,2,[<p>Hay una cosa que tienen en común estas dos...,12/11/2018,salud,Bella Hadid copió el truco de Naomi Campbell p...
3,3,[<p>Investigan si el expresidente falsificó fa...,12/11/2018,política,Piden que Amado Boudou vaya a juicio oral por ...
4,4,[<p>El temporal se sintió con fuerza en la pro...,12/11/2018,sociedad,Un tornado azotó el norte de Sante Fe


In [ ]:
texts = [TN, LPO, P12]

In [ ]:
stop_words_extra = []
stop_words = text.SPANISH_STOP_WORDS.union(stop_words_extra)

In [ ]:
# Genero lo vectores de las notas
count_vect = CountVectorizer(ngram_range = (1,3), max_df = 0.2, min_df = 0.01,stop_words=stop_words, lowercase=True)
x_counts = count_vect.fit_transform(texts)

In [ ]:
# Genero matriz con valorizacion tf-idf
tfidf_transformer = TfidfTransformer(norm = 'l2')
x_tfidf = tfidf_transformer.fit_transform(x_counts)

In [ ]:
# impongo el número de  tópicos
dim = 6

In [ ]:
#Realizo NMF
nmf = NMF(n_components = dim)
nmf_array = nmf.fit_transform(x_tfidf)

In [ ]:
# Labels de cada nota
labels = [np.argmax(x) for x in nmf_array]

In [ ]:
# Componentes y nombre de los feautures
components = [nmf.components_[i] for i in range(len(nmf.components_))]
features = count_vect.get_feature_names()

for j in range(len(components)):

    comp = components[j]

    prior_features = sorted(features, key = lambda x: comp[features.index(x)], reverse = True)

    # Guardo en archivos los features de cada componente ordenados por prioridad y 
    # las notas asociadas a cada tópico.
    fp = open('NMFComponent{}.txt'.format(j),'a')
    for k in prior_features:
        fp.write('{}, '.format(k))
    fp.close()
    
    
    fp = open('NMFNotes{}.txt'.format(j),'a')
    for k in range(len(labels)):
        if labels[k] == j:
            fp.write('{}, '.format(k))
    fp.close()